In [1]:
from Supporting_functions import *
from aicsimageio.readers import CziReader
from aicsimageio.writers import OmeTiffWriter
import re
import scipy.ndimage as ndi
from WBNS import WBNS_image
import RedLionfishDeconv as rl

/home/hmorales/miniconda3/envs/selfnet/lib/python3.9/site-packages/pydantic/_migration.py:281: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


# Crop and deconvolve czi images from a folder

In [2]:
## Predictions for a folder

##########################

# Image folder  to predict
img_src_path = '/run/user/1000/gvfs/smb-share:server=134.34.176.179,share=pmtest_fast/Daniel/231031_Sqt-sfGFP_Alexa647_H2AmCherry/' 

#output dir
outdir = '/media/hmorales/MyBookDuo/Data/IsoNet/Daniel_test/NoBG/'

##########################

# Experiment info
IsotropicOutput = 'Interpolation'       # 'No'
crop_images = True
thres_crop = 1.2
blur_mask = 2.0


# deconvolution
padding = 32
Niter = 5
psf_path = r'/media/hmorales/Skynet/IsoNet/Averaged_transformed_PSF_488.tif'


# PostPorcessing : BG subtraction and smoothing
resolution_px = 5 # FWHM of the PSF
noise_lvl = 2
sigma = 1.0

In [3]:
##########################
# Helpers

def open_image_from_reader(reader, color, order="ZYX", out_type=np.uint16):
    
    lazy_t0 = reader.get_image_dask_data(order, C=color)  # returns 3D ZYX numpy array
    img = lazy_t0.compute()  # returns in-memory 4D numpy array
    img = img.astype(out_type)
    
    return img, reader.physical_pixel_sizes.X, reader.physical_pixel_sizes.Z


def image_deconvolve(img0, psf, Niter, padding):
    
    # Padding image
    img = np.pad(img0, padding, mode='reflect')
    del img0
    imgSizeGB = img.nbytes / (1024 ** 3)
    print('     -Deconvolving size(GB) : ', imgSizeGB)
    # GPU deconvolution
    res_gpu = rl.doRLDeconvolutionFromNpArrays(img, psf, niter=Niter,resAsUint8=False)
    # Removing padding
    img = res_gpu[padding:-padding, padding:-padding, padding:-padding]
    
    return img


def custom_sort_images(file_name):
    allparts = file_name.split('(')
    parts = allparts[1].split(')')
    return (parts[0].zfill(4))


##########################  

# Create output folder
if not os.path.exists(outdir):
    os.mkdir(outdir)
      
# Open PSF and Prepare PSF
psf = tifffile.imread(psf_path)
psf_f = psf.astype(np.float32)
psf_norm = psf_f/psf_f.sum()




In [4]:
# get mask 

# Get all images in the folder
image_names = sorted([f for f in os.listdir(img_src_path) if f.endswith('.czi')])
image_names.sort(key=custom_sort_images)


N = len(image_names)
mN = int(0.5 * (N-1))

times4masking = [0, mN, N-1]

# get image path 
czi_file_path = os.path.join(img_src_path, image_names[0])   
reader = CziReader(czi_file_path)
raw_img, pixel_size_X, pixel_size_Z = open_image_from_reader(reader, 0, "ZYX", np.uint16)

# Sume up all channels

img = np.zeros_like(raw_img)
del raw_img

for idx in times4masking:
    czi_file_path = os.path.join(img_src_path, image_names[idx])   
    reader = CziReader(czi_file_path)
    print('Loading : ', image_names[idx])
    for color in range(reader.dims.C): 
        print("Processing color : ", str(color))
        raw_img, pixel_size_X, pixel_size_Z = open_image_from_reader(reader, color, "ZYX", np.uint16)
        img += raw_img
        del raw_img

Loading :  JackSkellington_2023_10_31__13_39_38_432_G2(25).czi
Processing color :  0
Processing color :  1
Processing color :  2
Loading :  JackSkellington_2023_10_31__13_39_38_432_G2(62).czi
Processing color :  0
Processing color :  1
Processing color :  2
Loading :  JackSkellington_2023_10_31__13_39_38_432_G2(99).czi
Processing color :  0
Processing color :  1
Processing color :  2


In [5]:
# get actual mask and boundary boxes

print(f"Getting tissue mask ...")
start_time = time.time()
        
scale = pixel_size_X / pixel_size_Z
if crop_images == True:
    bounds_min, bounds_max, mask = get_image_cropping_box(img, blur_mask, scale, thres_crop)                     
else:
    mask = get_image_simple_mask(img, blur_mask, scale, thres_crop)
    
Elapsed_time = time.time() - start_time
print(f"Elapsed Time masking: {Elapsed_time:.4f} seconds")      

Getting tissue mask ...
     -threshold_value: 2300.4
Elapsed Time masking: 175.5926 seconds


In [6]:
# Process all images

z,y,x = mask.shape
new_z = round(z / scale) 

for i, image_name in enumerate(image_names):
    
    print(f"Processing image : {image_name}")
    
    # get image path 
    czi_file_path = os.path.join(img_src_path, image_name)   
    reader = CziReader(czi_file_path)
    
    # Get time Id 
    match = re.search(r'\((\d+)\)', image_name)
    timeId = match.group(1)
    timeId= timeId.zfill(3)
    baseName= 'spim_TL'+str(timeId)

    for color in range(reader.dims.C): 
        print("Processing color : ", str(color))
        start_time = time.time()  # Record the start time
        # Open image
        raw_img, pixel_size_X, pixel_size_Z = open_image_from_reader(reader, color, "ZYX", np.uint16)
        scale = pixel_size_X / pixel_size_Z                 
        # crop image
        img = raw_img[bounds_min[0]:bounds_max[0], bounds_min[1]:bounds_max[1], bounds_min[2]:bounds_max[2]]
        del raw_img
        #make isotropic
        if IsotropicOutput == 'Interpolation':  
            img=reslice_bysize(img,'xy',new_z)
            pixel_sizeZ_new = pixel_size_X
            scale = 1.0
        else:
            pixel_sizeZ_new = pixel_size_Z
        # Deconvolution
        if Niter > 0 and IsotropicOutput != 'No':   
            img = image_deconvolve(img, psf, Niter, padding)
 
        # Remove BG
        if resolution_px > 0:
            img = WBNS_image(img, resolution_px, noise_lvl)

        # Smooth
        if sigma > 0:
            img = ndi.gaussian_filter(img, [sigma/scale, sigma, sigma])
           

        #save image
        outName = baseName+'_Channel'+str(color)+'.tif'        
        custom_save_img(img, outdir, outName, pixel_size_X,pixel_size_X, pixel_sizeZ_new)
        del img
        
        Elapsed_time = time.time() - start_time
        print(f"Elapsed Time: {Elapsed_time:.4f} seconds, image {outName}")  

Processing image : JackSkellington_2023_10_31__13_39_38_432_G2(25).czi
Processing color :  0
     -Deconvolving size(GB) :  4.189895838499069


Elapsed Time: 396.2347 seconds, image spim_TL025_Channel0.tif
Processing color :  1
     -Deconvolving size(GB) :  4.189895838499069


Elapsed Time: 280.3759 seconds, image spim_TL025_Channel1.tif
Processing color :  2
     -Deconvolving size(GB) :  4.189895838499069


Elapsed Time: 278.2942 seconds, image spim_TL025_Channel2.tif
Processing image : JackSkellington_2023_10_31__13_39_38_432_G2(26).czi
Processing color :  0
     -Deconvolving size(GB) :  4.189895838499069


Elapsed Time: 313.0311 seconds, image spim_TL026_Channel0.tif
Processing color :  1
     -Deconvolving size(GB) :  4.189895838499069


...

Elapsed Time: 305.5726 seconds, image spim_TL098_Channel1.tif
Processing color :  2
     -Deconvolving size(GB) :  4.189895838499069


Elapsed Time: 258.4703 seconds, image spim_TL098_Channel2.tif
Processing image : JackSkellington_2023_10_31__13_39_38_432_G2(99).czi
Processing color :  0
     -Deconvolving size(GB) :  4.189895838499069


Elapsed Time: 294.0445 seconds, image spim_TL099_Channel0.tif
Processing color :  1
     -Deconvolving size(GB) :  4.189895838499069


Elapsed Time: 286.7408 seconds, image spim_TL099_Channel1.tif
Processing color :  2
     -Deconvolving size(GB) :  4.189895838499069


Elapsed Time: 282.3224 seconds, image spim_TL099_Channel2.tif


In [7]:
'''
scale_img = reslice(raw_img,'xy',reader.physical_pixel_sizes.X,reader.physical_pixel_sizes.Z)            
beads_image =  img * (~mask)
beads_image = insert_predicted_image(scale_img,beads_image,bounds_min,bounds_max,1.0)


outName = 'spim_TL'+str(timeId)+'_Channel'+str(color+2)+'_Angle'+angles[view]+'.tif'        
custom_save_img(beads_image, outdir, outName, reader.physical_pixel_sizes.X, reader.physical_pixel_sizes.Y, reader.physical_pixel_sizes.Z)
'''


"\nscale_img = reslice(raw_img,'xy',reader.physical_pixel_sizes.X,reader.physical_pixel_sizes.Z)            \nbeads_image =  img * (~mask)\nbeads_image = insert_predicted_image(scale_img,beads_image,bounds_min,bounds_max,1.0)\n\n\noutName = 'spim_TL'+str(timeId)+'_Channel'+str(color+2)+'_Angle'+angles[view]+'.tif'        \ncustom_save_img(beads_image, outdir, outName, reader.physical_pixel_sizes.X, reader.physical_pixel_sizes.Y, reader.physical_pixel_sizes.Z)\n"